In [43]:
# 라이브러리 import
from neo4j import GraphDatabase
import re

# Neo4j 연결 정보 (너의 환경에 맞게 수정)
uri = "bolt://localhost:7687"  # 기본 포트
username = "neo4j"
password = "yoonca51"  # 네가 설정한 비밀번호

# Neo4j 연결 드라이버 생성
driver = GraphDatabase.driver(uri, auth=(username, password))

# Neo4j에서 entity 목록을 한번만 가져와 메모리에 캐싱하는 함수
def load_all_entities_from_neo4j():
    entities_set = set()
    with driver.session() as session:
        query = "MATCH (e:Entity) RETURN DISTINCT e.name as name"
        results = session.run(query)
        for record in results:
            entities_set.add(record["name"].lower())
    return entities_set

# 캐싱된 entity 목록
cached_entities = load_all_entities_from_neo4j()

# 입력 텍스트에서 FinDKG의 entity만 효율적으로 식별하는 함수
def identify_entities_in_text(T, cached_entities):
    identified_entities = set()
    T_lower = T.lower() #  입력 텍스트를 소문자로 변환

    for entity in cached_entities:
        pattern = r'\b' + re.escape(entity.lower()) + r'\b'
        if re.search(pattern, T_lower):
            identified_entities.add(entity)

    return identified_entities

# 식별된 entity를 바탕으로 Neo4j에서 외부지식 N(e)를 추출하는 함수

def extract_external_knowledge(T, cached_entities,time : str):
    # time format : 
    entities = identify_entities_in_text(T, cached_entities)
    
    knowledge = {}
    
    with driver.session() as session:
        for entity in entities:
            query = """
            MATCH (e:Entity {name: $entity})-[r]->(neighbor)
            WHERE r.time < $time
            RETURN r.type as relation,r.time as time, neighbor.name as neighbor_entity
            """
            results = session.run(query, entity=entity , time = time)
            
            knowledge[entity] = []
            for record in results:
                knowledge[entity].append({
                    "relation": record["relation"],
                    "neighbor_entity": record["neighbor_entity"],
                    "time": record["time"]
                })
    
    return knowledge

In [44]:
T_example = """
    President Trump Administration had an influence on the Volcker rule. 
    Wells Fargo Co. also impacted U.S. Federal Reserve policies.
    """
external_knowledge = extract_external_knowledge(T_example, cached_entities,'2018-06-03')
print("Extracted Knowledge:", external_knowledge)

Extracted Knowledge: {'.': [], 'volcker rule': [], 'policies': [], 'u.s. federal reserve': [{'relation': 'Relate_To', 'neighbor_entity': 'us government', 'time': '2018-03-18'}, {'relation': 'Operate_In', 'neighbor_entity': 'us government', 'time': '2018-02-04'}, {'relation': 'Operate_In', 'neighbor_entity': 'us government', 'time': '2018-05-27'}, {'relation': 'Relate_To', 'neighbor_entity': 'donald trump', 'time': '2018-01-07'}, {'relation': 'Relate_To', 'neighbor_entity': 'donald trump', 'time': '2018-01-07'}, {'relation': 'Relate_To', 'neighbor_entity': 'donald trump', 'time': '2018-01-14'}, {'relation': 'Relate_To', 'neighbor_entity': 'united states', 'time': '2018-04-15'}, {'relation': 'Operate_In', 'neighbor_entity': 'united states', 'time': '2018-05-06'}, {'relation': 'Operate_In', 'neighbor_entity': 'united states', 'time': '2018-05-06'}, {'relation': 'Has', 'neighbor_entity': 'united states', 'time': '2018-01-14'}, {'relation': 'Operate_In', 'neighbor_entity': 'united states', 

In [51]:
for x in external_knowledge:
    print(x)
    for y in external_knowledge[x]:
        print(y)

.
volcker rule
policies
u.s. federal reserve
{'relation': 'Relate_To', 'neighbor_entity': 'us government', 'time': '2018-03-18'}
{'relation': 'Operate_In', 'neighbor_entity': 'us government', 'time': '2018-02-04'}
{'relation': 'Operate_In', 'neighbor_entity': 'us government', 'time': '2018-05-27'}
{'relation': 'Relate_To', 'neighbor_entity': 'donald trump', 'time': '2018-01-07'}
{'relation': 'Relate_To', 'neighbor_entity': 'donald trump', 'time': '2018-01-07'}
{'relation': 'Relate_To', 'neighbor_entity': 'donald trump', 'time': '2018-01-14'}
{'relation': 'Relate_To', 'neighbor_entity': 'united states', 'time': '2018-04-15'}
{'relation': 'Operate_In', 'neighbor_entity': 'united states', 'time': '2018-05-06'}
{'relation': 'Operate_In', 'neighbor_entity': 'united states', 'time': '2018-05-06'}
{'relation': 'Has', 'neighbor_entity': 'united states', 'time': '2018-01-14'}
{'relation': 'Operate_In', 'neighbor_entity': 'united states', 'time': '2018-01-21'}
{'relation': 'Operate_In', 'neighbo